## Scalable and Cost-Effective Deduplication: Leveraging Algorithms and LLMs

### By: Rohan Chaudhary, Sarmad Kahut, Valentina Torres


The presented Jupyter notebook focuses exclusively on the states of Florida and Nevada as the selected testing states for the deduplication challenge. These states were chosen for their specific characteristics and relevance to the task. However, it's important to note that this is just a subset of the complete set of states. To ensure comprehensive results, the user from SponsorMotion is encouraged to replicate the provided code and model with the remaining states in the dataset. 

### Data Cleaning, Sorting and Standardization 

In [1]:
import pandas as pd
import numpy as np

In [2]:
#data = pd.read_csv('ordered.csv')
data = pd.read_csv('230607 Events dump.csv')
data.shape

/var/folders/50/4pcz1qlx5vg004krks9c5vsm0000gn/T/ipykernel_48445/2455299057.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('230607 Events dump.csv')


(48265, 21)

In [3]:
data=data.dropna(subset=['name','start date','state'],how='all')

In [4]:
data.shape

(45011, 21)

After dropping all the rows that have no name, no start date and no state, the dataset resulted in 45,011 records. 

In [5]:
data.dtypes

country                object
end date               object
flagged_by             object
has_embedding          object
human verification     object
is virtual             object
is_flagged             object
is_free                object
location               object
name                   object
sourceURL              object
specialties            object
start date             object
state                  object
summary                object
URL                   float64
Creation Date          object
Modified Date          object
Slug                  float64
Creator                object
unique id              object
dtype: object

The following code is used to reformat the "start date" column into the YYYY/MM/DD format. This adjustment is crucial as it greatly facilitates data sorting and enables efficient tracking of duplicates. By standardizing the start date representation, it becomes an important criterion for grouping potential duplicates accurately.

In [6]:
data['start date'] = pd.to_datetime(data['start date'], format='%b %d, %Y %I:%M %p', errors='coerce').dt.strftime('%Y/%m/%d')
data['start date'] = pd.to_datetime(data['start date'])

We are adding a new column that shows the summary length in terms of the number of words. Moreover, we will retain the initial column, which corresponds to the longest summary. By doing this we want to prioritize rows with the most comprehensive event information ensuring that we preserve entries with richer details, enhancing the overall data quality and accuracy.

In [7]:
data['summary_len']=data['summary'].apply(lambda x: len(x) if isinstance(x, str) else None)

In [8]:
data['summary_len'][1:10]

1    469.0
2    487.0
3    742.0
4    247.0
5    364.0
6    491.0
7    207.0
8    505.0
9    487.0
Name: summary_len, dtype: float64

In [9]:
def order_dataset(data):
    data.sort_values(by=['start date', 'state','summary_len'],ascending=[True,True,False] ,inplace=True)
    data.reset_index(drop=True, inplace=True)
    return data

ordered_data = order_dataset(data)

Below, we are customizing the "name" column. Our process involves eliminating all spaces and special characters, followed by converting all text to lowercase. This tailored approach serves the purpose of maintaining a standardized format for the information. By carrying out these modifications, we'll enhance the effectiveness of the fuzzy matching algorithm, enabling it to more efficiently identify potential duplicates. This strategic adjustment contributes to a more precise deduplication process and overall data quality.

In [10]:
#remove all the initial and ending spaces
ordered_data['name']=ordered_data['name'].str.strip()

In [11]:
#removing # and / from text
ordered_data['name'] = ordered_data['name'].str.replace(r'[#/\$()-;Ä+¬Æ¬†¬∞Ñ¢ì @!≤√]', '', regex=True)

In [12]:
ordered_data['name'] = ordered_data['name'].str.replace(' ', '', regex=True)

In [13]:
ordered_data['name']=ordered_data['name'].str.lower()

In [14]:
ordered_data['name'][1:5]

1                chickasawconstitutiondrafting
2                        thebattleofgettysburg
3        feministoratorwowsterritorialcheyenne
4    americangynecologicalsocietyannualmeeting
Name: name, dtype: object

Now we pick the states for which we want to analyze the information. In this notebook we will use Florida and Nevada which are 2 of the states with the highest number of records. 

In [15]:
filtered_data = ordered_data[ordered_data['state'].isin(['FL','NV'])]# Change the state name accroding to your desire.

In [16]:
filtered_data.shape

(1772, 22)

### Deduplication Algorithm

We will first filter all the records that have same name, start date and state since they are potential duplicates and only keep the first one which is the one that has the longest summary. After the otehr records are dropped, we will end up with a dataset that contains unique events. 

In [20]:
# keeping unique records of names start date and state
unique_events= filtered_data.drop_duplicates(['name','start date','state'], keep='first')

In [21]:
unique_events.drop(columns=['summary_len'], inplace=True)

/var/folders/50/4pcz1qlx5vg004krks9c5vsm0000gn/T/ipykernel_48445/4027050285.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_events.drop(columns=['summary_len'], inplace=True)


While these records are distinct, some could potentially be duplicates, differing by just +1 or -1 days in their start dates. There's a chance they share identical names coupled with similar start dates, or slightly differing names alongside comparable or even identical start dates. To address this, we'll employ a fuzzy matching algorithm using the name column. This technique will effectively flag potential duplicates, taking into account the subtle variations in names and dates.

In [22]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

When we run the algorithm we will use a treshold of 75 which has shown the highest accuracy during testing and less amount of false negatives. 

In [23]:
from fuzzywuzzy import fuzz
import pandas as pd

# Define a function to check fuzzy matching similarity
def fuzzy_match_similarity(name_a, name_b):
    return fuzz.token_set_ratio(name_a, name_b)

# Apply fuzzy matching on all name pairs within each state group
potential_duplicates = []

# Group by 'state' column
grouped_events = unique_events.groupby('state')

for group_name, group_data in grouped_events:
    # Sort the group_data DataFrame by 'start date'
    group_data = group_data.sort_values(by='start date')
    
    for i in range(len(group_data)):
        for j in range(i + 1, len(group_data)):
            name_a = group_data.iloc[i]['name']
            name_b = group_data.iloc[j]['name']
            formatted_date_a = pd.to_datetime(group_data.iloc[i]['start date'])
            formatted_date_b = pd.to_datetime(group_data.iloc[j]['start date'])
            
            # Check if the dates are the same or differ by one day
            date_difference = abs((formatted_date_a - formatted_date_b).days)
            
            if date_difference <= 1:  # Events are either on the same date or differ by one day
                similarity = fuzzy_match_similarity(name_a, name_b)
                if similarity >= 75:
                    potential_duplicates.append((group_data.iloc[i], group_data.iloc[j]))

We're currently refining the identification of potential duplicates by introducing a new column titled "flagged_duplicates_numbers." This column assigns a consistent number to all rows that qualify as potential duplicates.

In [25]:
from fuzzywuzzy import fuzz
import pandas as pd
# Define a function to check fuzzy matching similarity
def fuzzy_match_similarity(name_a, name_b):
    return fuzz.token_set_ratio(name_a, name_b)
# Assuming you have already obtained the 'potential_duplicates' list
# Create a list to store sets of duplicate name and start date combinations
duplicate_groups = []
for pair in potential_duplicates:
    row_a, row_b = pair
    name_a = row_a['name']
    date_a = row_a['start date']
    name_b = row_b['name']
    date_b = row_b['start date']
    # Find if either name or date exists in any existing group
    found_group_index = None
    for i, group in enumerate(duplicate_groups):
        if (name_a, date_a) in group or (name_b, date_b) in group:
            found_group_index = i
            break
    # If both name and date are new, create a new group
    if found_group_index is None:
        duplicate_groups.append({(name_a, date_a), (name_b, date_b)})
    else:
        # Add the new name and date to the existing group
        duplicate_groups[found_group_index].add((name_a, date_a))
        duplicate_groups[found_group_index].add((name_b, date_b))
# Create a dictionary to map each unique name and start date combination to a numerical value
name_date_to_numerical = {}
numerical_value = 1
for group in duplicate_groups:
    for name, date in group:
        name_date_to_numerical[(name, date)] = numerical_value
    numerical_value += 1
# Create a new column 'flagged_duplicates_numbers' in the original dataset
unique_events['flagged_duplicates_numbers'] = 0
# Iterate through the rows and set the numerical values for potential duplicates
for index, row in unique_events.iterrows():
    name = row['name']
    date = row['start date']
    if (name, date) in name_date_to_numerical:
        unique_events.at[index, 'flagged_duplicates_numbers'] = name_date_to_numerical[(name, date)]


/var/folders/50/4pcz1qlx5vg004krks9c5vsm0000gn/T/ipykernel_48445/2100794347.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_events['flagged_duplicates_numbers'] = 0


Finally, we add an extra column "human_verification_needed" this will indicate if that record needs further verification. It will appear as "yes" if the records have similar names but different start dates (+ or - 1 day) and it will appear as "no" if the records have similar names and same dates.

In [26]:
# Assuming you already have the DataFrame 'unique_events' with the 'flagged_duplicates_numbers' column
# Create a new column 'human_verification_needed' and initialize it with 'No'
unique_events['human_verification_needed'] = 'No'
# Iterate through the rows and update 'human_verification_needed' based on the conditions
for index, row in unique_events.iterrows():
    flagged_num = row['flagged_duplicates_numbers']
    if flagged_num != 0:
        same_group_rows = unique_events[unique_events['flagged_duplicates_numbers'] == flagged_num]
        start_date = row['start date']
        # Collect unique 'start_date' values for the group
        unique_dates = set(same_group_rows['start date'])
        unique_dates.remove(start_date)  # Remove the current 'start_date' to avoid checking against itself
        # Check if 'start_date' differs by exactly one day for any other row in the same group
        for date in unique_dates:
            if pd.Timedelta(days=-1) == date - start_date or pd.Timedelta(days=1) == date - start_date:
                unique_events.at[index, 'human_verification_needed'] = 'Yes'
                break
# Print the updated dataset with the new column
#print(unique_events)
unique_events.to_csv('FL+NV_75.csv',index=False)

/var/folders/50/4pcz1qlx5vg004krks9c5vsm0000gn/T/ipykernel_48445/1996621375.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_events['human_verification_needed'] = 'No'
